In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
import string
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('dataset/tweetlabels1000_labeled.xlsx - Sheet2.csv')
print(df.shape)
df

(1003, 4)


,No,Label,Username,Tweet
0,0,none,chyrisalys,wifi watcha pasti indihome
1,1,indirect complaint,woiidal,indihome ada masalah apasih??!!
2,2,remark,ranieaw,sore sore hujan rebahan bareng bocil nonton nu...
3,3,negative remark,daeguv_,"indihome plis untuk tanggal 10,13 jangan kesur..."
4,4,indirect complaint,fiorincha,INDIHOME NGAPASIIII
...,...,...,...,...
998,998,campaign,sangwarior,Bisa sharing bareng sobat indihome gini kan as...
999,999,campaign,sangwarior,Sekarang nyantai dulu bentar sambil scroll sos...
1000,1000,negative remark,untextend,@JefriHandri Sini indihome down dr semalem jam...
1001,1001,campaign,sangwarior,Ini barusan slesei sob... Lumayan buat nyari k...


In [3]:
# drop rows with label = campaign, own tweet, & incomplete
df = df.loc[~df['Label'].isin(['campaign','own tweet','incomplete'])].copy()
df

,No,Label,Username,Tweet
0,0,none,chyrisalys,wifi watcha pasti indihome
1,1,indirect complaint,woiidal,indihome ada masalah apasih??!!
2,2,remark,ranieaw,sore sore hujan rebahan bareng bocil nonton nu...
3,3,negative remark,daeguv_,"indihome plis untuk tanggal 10,13 jangan kesur..."
4,4,indirect complaint,fiorincha,INDIHOME NGAPASIIII
...,...,...,...,...
994,994,inquiry,pecintamochi,"@IndiHomeCare Min, kalo jatuh tempo pembayaran..."
995,995,inquiry,tetehaisyah51,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl..."
997,997,direct complaint,untextend,@IndiHomeCare @fauzindrianto Dari semalem down...
1000,1000,negative remark,untextend,@JefriHandri Sini indihome down dr semalem jam...


In [4]:
# Label Encoding & drop columns
df['label'] = df['Label'].map({'indirect complaint': 0, 'remark': 1, 'negative remark': 2, 'direct compliment' : 3 , 'direct complaint' : 4, 'none' : 5, 'inquiry' : 6})
df.drop(['No', 'Label', 'Username'], axis=1, inplace=True)
df

,Tweet,label
0,wifi watcha pasti indihome,5
1,indihome ada masalah apasih??!!,0
2,sore sore hujan rebahan bareng bocil nonton nu...,1
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2
4,INDIHOME NGAPASIIII,0
...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4
1000,@JefriHandri Sini indihome down dr semalem jam...,2


In [5]:
df.groupby( by='label').count()

,Tweet
label,
0,68
1,153
2,128
3,64
4,110
5,123
6,76


In [6]:
# to lowercase
df['tweet_lower'] = df['Tweet'].str.lower()
df

,Tweet,label,tweet_lower
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur..."
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii
...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran..."
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl..."
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...


In [7]:
def text_cleansing(text):
    
       
    # remove non ASCII (emoticon, chinese word, etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    
    # remove digits (using regex) -> subtitute
    text = re.sub('\d+', '', text)
    
    # remove punctuation, reference: https://stackoverflow.com/a/34294398
    #text = text.translate(str.maketrans('', '', string.punctuation))
    
    # remove punctuation, except mention tag @
    text = re.sub(r'[^\w\s@]', '', text)
    
    # remove whitespace in the beginning and end of sentence
    text = text.strip()
    
    # remove extra whitespace in the middle of sentence (using regex)
    text = re.sub('\s+', ' ', text)
    
    # remove url in tweet (using regex)
    text = re.sub(r"\bhttp\w+", "", text)
    
    return text

In [8]:
# text_cleansing()
df['tweet_clean'] = df['tweet_lower'].apply(lambda x: text_cleansing(x))
df

,Tweet,label,tweet_lower,tweet_clean
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!,indihome ada masalah apasih
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocil nonton nu...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur...",indihome plis untuk tanggal jangan kesurupan
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii,indihome ngapasiiii
...,...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran...",@indihomecare min kalo jatuh tempo pembayaran ...
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl...",@indihomecare min indihome tv bisa gak berlang...
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...,@indihomecare @fauzindrianto dari semalem down...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...,@jefrihandri sini indihome down dr semalem jam...


In [9]:
# normalization indonesian words
normalized_word = pd.read_csv('dataset/new_kamusalay.csv', header=None)
data_dict = dict(zip(normalized_word[0], normalized_word[1]))
len(data_dict)

15166

In [10]:
def normalize_text(text):
    return ' '.join(data_dict.get(word, word) for word in text.split())

In [11]:
df['tweet_normalized'] = df['tweet_clean'].apply(lambda x: normalize_text(x))
df

,Tweet,label,tweet_lower,tweet_clean,tweet_normalized
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome,wifi watcha pasti indihome,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!,indihome ada masalah apasih,indihome ada masalah apa sih
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocah cilik men...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur...",indihome plis untuk tanggal jangan kesurupan,indihome tolong untuk tanggal jangan kesurupan
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii,indihome ngapasiiii,indihome ngapasiiii
...,...,...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran...",@indihomecare min kalo jatuh tempo pembayaran ...,@indihomecare min kalau jatuh tempo pembayaran...
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl...",@indihomecare min indihome tv bisa gak berlang...,@indihomecare min indihome televisi bisa tidak...
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...,@indihomecare @fauzindrianto dari semalem down...,@indihomecare @fauzindrianto dari semalam down...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...,@jefrihandri sini indihome down dr semalem jam...,@jefrihandri sini indihome down dari semalam j...


In [12]:
# stemming 'kata berimbuhan'
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [13]:
df['tweet_stemmed'] = df['tweet_normalized'].apply(lambda x: stemmer.stem(x))
df

,Tweet,label,tweet_lower,tweet_clean,tweet_normalized,tweet_stemmed
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome,wifi watcha pasti indihome,wifi watcha pasti indihome,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!,indihome ada masalah apasih,indihome ada masalah apa sih,indihome ada masalah apa sih
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocah cilik men...,sore sore hujan rebah bareng bocah cilik tonto...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur...",indihome plis untuk tanggal jangan kesurupan,indihome tolong untuk tanggal jangan kesurupan,indihome tolong untuk tanggal jangan surup
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii,indihome ngapasiiii,indihome ngapasiiii,indihome ngapasiiii
...,...,...,...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran...",@indihomecare min kalo jatuh tempo pembayaran ...,@indihomecare min kalau jatuh tempo pembayaran...,indihomecare min kalau jatuh tempo bayar indih...
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl...",@indihomecare min indihome tv bisa gak berlang...,@indihomecare min indihome televisi bisa tidak...,indihomecare min indihome televisi bisa tidak ...
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...,@indihomecare @fauzindrianto dari semalem down...,@indihomecare @fauzindrianto dari semalam down...,indihomecare fauzindrianto dari malam down tid...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...,@jefrihandri sini indihome down dr semalem jam...,@jefrihandri sini indihome down dari semalam j...,jefrihandri sini indihome down dari malam jam ...


In [14]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [15]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=7, activation='sigmoid')
    ])
    
    model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [16]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils.np_utils import to_categorical

# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(n_splits=10, shuffle=False, random_state=None)

# Separate the tweet and label
tweets, labels = list(df.tweet_stemmed), list(df.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(tweets):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(tweets[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(tweets[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # one-hot
    onehot_train_y = to_categorical(train_y)
    onehot_test_y = to_categorical(test_y)

    # encode data using Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, onehot_train_y, batch_size=32, epochs=30, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, onehot_test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, onehot_test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/30
21/21 [==============================] - 15s 262ms/step - loss: 1.8975 - accuracy: 0.2696 - val_loss: 1.9817 - val_accuracy: 0.0959
Epoch 2/30
21/21 [==============================] - 1s 70ms/step - loss: 1.6851 - accuracy: 0.3590 - val_loss: 1.8659 - val_accuracy: 0.2740
Epoch 3/30
21/21 [==============================] - 1s 70ms/step - loss: 1.2363 - accuracy: 0.5593 - val_loss: 1.8007 - val_accuracy: 0.3288
Epoch 4/30
21/21 [==============================] - 1s 69ms/step - loss: 0.7181 - accuracy: 0.7550 - val_loss: 2.0322 - val_accuracy: 0.2740
Epoch 5/30
21/21 [==============================] - 1s 70ms/step - loss: 0.4080 - accuracy: 0.8706 - val_loss: 2.3002 - val_accuracy: 0.2603
Epoch 6/30
21/21 [==============================] - 1s 68ms/step - loss: 0.2231 - accuracy: 0.9353 - val_loss: 2.6564 - val_accuracy: 0.2740
Epoch 7/30
21/21 [==============================] - 2s 72ms/step - loss: 0.1500 - accuracy: 0.9630 - val_loss: 2.9069 - val_accuracy: 0.301

21/21 [==============================] - 3s 131ms/step - loss: 0.0219 - accuracy: 0.9938 - val_loss: 3.4678 - val_accuracy: 0.4444
Epoch 17/30
21/21 [==============================] - 3s 131ms/step - loss: 0.0270 - accuracy: 0.9938 - val_loss: 3.3675 - val_accuracy: 0.4722
Epoch 18/30
21/21 [==============================] - 3s 130ms/step - loss: 0.0270 - accuracy: 0.9908 - val_loss: 3.5726 - val_accuracy: 0.4583
Epoch 19/30
21/21 [==============================] - 3s 130ms/step - loss: 0.0334 - accuracy: 0.9892 - val_loss: 3.5467 - val_accuracy: 0.4583
Epoch 20/30
21/21 [==============================] - 3s 130ms/step - loss: 0.0241 - accuracy: 0.9954 - val_loss: 3.5603 - val_accuracy: 0.4722
Epoch 21/30
21/21 [==============================] - 3s 129ms/step - loss: 0.0247 - accuracy: 0.9938 - val_loss: 3.5539 - val_accuracy: 0.4444
Epoch 22/30
21/21 [==============================] - 3s 130ms/step - loss: 0.0235 - accuracy: 0.9908 - val_loss: 3.5585 - val_accuracy: 0.4583
Epoch 23/30

Epoch 6/30
21/21 [==============================] - 3s 140ms/step - loss: 0.2428 - accuracy: 0.9308 - val_loss: 1.9064 - val_accuracy: 0.3750
Epoch 7/30
21/21 [==============================] - 3s 141ms/step - loss: 0.1301 - accuracy: 0.9677 - val_loss: 2.0406 - val_accuracy: 0.3750
Epoch 8/30
21/21 [==============================] - 3s 141ms/step - loss: 0.0939 - accuracy: 0.9677 - val_loss: 2.2995 - val_accuracy: 0.4167
Epoch 9/30
21/21 [==============================] - 3s 142ms/step - loss: 0.0684 - accuracy: 0.9862 - val_loss: 2.2937 - val_accuracy: 0.3750
Epoch 10/30
21/21 [==============================] - 3s 142ms/step - loss: 0.0535 - accuracy: 0.9892 - val_loss: 2.5605 - val_accuracy: 0.3611
Epoch 11/30
21/21 [==============================] - 3s 142ms/step - loss: 0.0673 - accuracy: 0.9815 - val_loss: 2.6309 - val_accuracy: 0.3611
Epoch 12/30
21/21 [==============================] - 3s 143ms/step - loss: 0.0661 - accuracy: 0.9815 - val_loss: 2.6208 - val_accuracy: 0.4028
Epo

21/21 [==============================] - 3s 148ms/step - loss: 0.0319 - accuracy: 0.9908 - val_loss: 3.7072 - val_accuracy: 0.3889
Epoch 20: early stopping
Test Accuracy: 44.44444477558136

       acc1       acc2  acc3       acc4       acc5      acc6      acc7  \
0  35.61644  41.095892  50.0  43.055555  44.444445  48.61111  51.38889   

        acc8       acc9      acc10        AVG  
0  56.944442  34.722221  44.444445  45.032344  



C:\Users\baktistr\AppData\Local\Temp\ipykernel_66428\3147406012.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record = record.append(temp, ignore_index=True)


In [18]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,35.61644,41.095892,50.0,43.055555,44.444445,48.61111,51.38889,56.944442,34.722221,44.444445,45.032344


In [19]:
# define the preprocessing pipeline
def text_preprocessing(text):
    # case folding
    text = text.lower()
    
    # text cleansing
    text = text_cleansing(text)
    
    # text normalization
    text = normalize_text(text)
    
    # stemming
    text = stemmer.stem(text)
    
    return text

In [20]:
from keras.utils.np_utils import to_categorical

# unseen tweet
new_tweet = "@IndiHome internet saya lemot sudah 3 hari"

# preprocess
text_cleaned = text_preprocessing(new_tweet)
text_sequence = tokenizer.texts_to_sequences([text_cleaned])

max_len= max_length(training_sequences)

seq_padded = pad_sequences(text_sequence, maxlen=max_len, padding=padding_type, truncating=trunc_type)

# prediction
pred_proba = model.predict(seq_padded)

result = np.argmax(pred_proba)
result = {
    0: lambda result: "indirect complaint",
    1: lambda result: "remark",
    2: lambda result: "negative remark",
    3: lambda result: "direct compliment",
    4: lambda result: "direct complaint",
    5: lambda result: "none",
    6: lambda result: "inquiry"
}[result](result)

print(result)

direct complaint
